# Packages

In [24]:
from ortools.sat.python import cp_model
from ortools.constraint_solver import pywrapcp

import sys
import time

import os
os.getcwd()

'c:\\Users\\gilramolete\\OneDrive - UNIONBANK of the Philippines\\Documents 1\\Route Optimization\\OR-Tools'

# Constraint Optimization

## Overview

**Constraint optimization**, or **constraint programming** (CP), is the name given to identifying feasible solutions out of a **very large set of candidates**, where the problem can be modeled in terms of arbitrary constraints. CP problems arise in many scientific and engineering disciplines. (The word "programming" is a bit of a misnomer, similar to how "computer" once meant "a person who computes". Here, "programming" refers to the *arrangement of a plan*, rather than programming in a computer language.)

CP is based on *feasibility* (finding a feasible solution) rather than optimization (finding an optimal solution) and focuses on the constraints and variables rather than the objective function. In fact, a CP problem may not even have an objective function — the goal may simply be to narrow down a very large set of possible solutions to a more manageable subset by adding constraints to the problem.

An example of a problem that is well-suited for CP is **employee scheduling**. The problem arises when companies that operate continuously — such as factories — need to create weekly schedules for their employees. Here's a very simple example: a company runs three 8-hour shifts per day and assigns three of its four employees to different shifts each day, while giving the fourth the day off. Even in such a small case, the number of possible schedules is huge: each day, there are `4! = 4 * 3 * 2 * 1 = 24` possible employee assignments, so the number of possible weekly schedules is 247, which is over 4.5 billion. Usually there will be other constraints that reduce the number of feasible solutions — for example, that each employee works at least a minimum number of days per week. The CP method keeps track of which solutions remain feasible when you add new constraints, which makes it a powerful tool for solving large, real-world scheduling problems.

CP has a widespread and very active community around the world with dedicated scientific journals, conferences, and an arsenal of different solving techniques. CP has been successfully applied in planning, scheduling, and numerous other domains with heterogeneous constraints.

### Tools

Google provides few ways to solve CP problems:
- [CP-SAT solver](https://developers.google.com/optimization/cp/cp_solver): A **constraint programming** solver that uses SAT (satisfiability) methods.
- [Original CP solver](https://developers.google.com/optimization/cp/original_cp_solver): A **constraint programming** solver.

If your problem can be modeled with a linear objective and linear constraints, then you have a linear programming problem and should consider [MPSolver](https://developers.google.com/optimization/lp/mpsolver).

## CP-SAT Solver

Let's start with a simple example problem in which there are:
- Three variables, x, y, and z, each of which can take on the values: 0, 1, or 2.
- One constraint: `x != y`

We'll start by showing how to use the CP-SAT solver to find a single feasible solution in all of the supported languages. While finding a feasible solution is trivial in this case, in more complex constraint programming problems it can be very difficult to determine whether there is a feasible solution.

### Finding a feasible solution

In [5]:
# Declare the model
model = cp_model.CpModel()

# Create variables
num_vals = 3
x = model.NewIntVar(0, num_vals - 1, 'x')
y = model.NewIntVar(0, num_vals - 1, 'y')
z = model.NewIntVar(0, num_vals - 1, 'z')
# each can take values 0, 1, or 2

# Create the constraint
model.Add(x != y)

# Call solver
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Display the first solution
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('x = %i' % solver.Value(x))
    print('y = %i' % solver.Value(y))
    print('z = %i' % solver.Value(z))
else:
    print('No solution found.')

x = 1
y = 0
z = 0


The CP-SAT solver returns one of the status values shown in the table below. In this example, the value returned is `OPTIMAL`.

| Status | Description |
| --- | --- |
| `OPTIMAL` | An optimal feasible solution was found |
| `FEASIBLE` | A feasible solution was found, but we don't know if it's optimal |
| `INFEASIBLE` | The problem was proven infeasible |
| `MODEL_INVALID` | The given `CpModelProto` didn't past the valudation step. You can get a detailed error by calling `ValidateCpModel(model_proto)` |
| `UNKNOWN` | The status of the model is unknown because no solution was found (or the problem was not proven `INFEASIBLE`) before something caused the solver to stop, such as a time limit, a memory limit, or a custom limit set by the user. |

### Finding all solutions

Next, we'll show how to modify the program above to find all feasible solutions.

The main addition to the program is a **solution printer** a callback that you pass to the solver, which displays each solution as it is found.

In [8]:
# Add the solution printer
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    '''Print intermediate solutions.'''

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0
    
    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print('%s = %i' % (v, self.Value(v)), end = '   ')
        print()

    def solution_count(self):
        return self.__solution_count

In [9]:
# Call solver
solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter([x, y, z])

# Enumerate all solutions
solver.parameters.enumerate_all_solutions = True

# Solve
status = solver.Solve(model, solution_printer)

# Display solution
print('Status = %s' % solver.StatusName(status))
print('Number of solutions found: %i' % solution_printer.solution_count())

x = 1   y = 0   z = 0   
x = 2   y = 0   z = 0   
x = 2   y = 0   z = 1   
x = 1   y = 0   z = 1   
x = 2   y = 1   z = 1   
x = 2   y = 1   z = 0   
x = 2   y = 1   z = 2   
x = 2   y = 0   z = 2   
x = 1   y = 0   z = 2   
x = 0   y = 1   z = 2   
x = 0   y = 1   z = 1   
x = 0   y = 2   z = 1   
x = 0   y = 2   z = 2   
x = 1   y = 2   z = 2   
x = 1   y = 2   z = 1   
x = 1   y = 2   z = 0   
x = 0   y = 2   z = 0   
x = 0   y = 1   z = 0   
Status = OPTIMAL
Number of solutions found: 18


## Solving a CP Problem

Maximize 2x + 2y + 3z subject to the following constraints:
$$x + 7⁄2 y + 3⁄2 z	≤	25$$
$$3x - 5y + 7z	≤	45$$
$$5x + 2y - 6z	≤	37$$
$$x, y, z	≥	0$$
$$x, y, z \text{   integers}$$

In order to increase computational speed, the CP-SAT solver works over the integers. This means all constraints and the objective must have integer coefficients. In the above example, the first constraint doesn't meet this condition. To solve the problem, you must first transform the constraint by multiplying it by a sufficiently large integer to convert all the coefficients to integers.

Since the first constraint, $x + 7⁄2 y + 3⁄2 z	≤	25$, has non-integer coefficients, you must first multiply the entire constraint by a sufficiently large integer to convert all the coefficients to integers. In this case, we can multiply by 2, which results in the new constraint $x + 7y +  3z	≤	50$. This doesn't change the problem, since the original constraint has exactly the same solutions as the transfomed constraints.

In [12]:
# Declare model
model = cp_model.CpModel()

# Create variables
var_upper_bound = max(50, 45, 37)
x = model.NewIntVar(0, var_upper_bound, 'x')
y = model.NewIntVar(0, var_upper_bound, 'y')
z = model.NewIntVar(0, var_upper_bound, 'z')

# Define the constraints
model.Add(2 * x + 7 * y + 3 * z <= 50)
model.Add(3 * x - 5 * y + 7 * z <= 45)
model.Add(5 * x + 2 * y - 6 * z <= 37)

# Define objective
model.Maximize(2 * x + 2 * y + 3 * z)

# Call solver
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Display solution
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f'Maximum of objective function: {solver.ObjectiveValue()}\n')
    print(f'x = {solver.Value(x)}')
    print(f'y = {solver.Value(y)}')
    print(f'z = {solver.Value(z)}')
else:
    print('No solution found.')

Maximum of objective function: 35.0

x = 7
y = 3
z = 5


## Cryptarithmetic Puzzles

A *cryptarithmetic puzzle* is a mathematical exercise where the digits of some numbers are represented by letters (or symbols). Each letter represents a unique digit. The goal is to find the digits such that a given mathematical equation is verified:

        CP
    +   IS
    +   FUN
    -------
    =   TRUE

One assignment of letters to digits yields the following equation:

        23
    +   74
    +   968
    -------
    =   1065

There are other answers to this problem.

### Modeling the problem

As with any optimization problem, we'll start by identifying variables and constraints. The variables are the letters, which can take on any single digit value.

For CP + IS + FUN = TRUE, the constraints are as follows:
- The equation: `CP + IS + FUN = TRUE`.
- Each of the ten letters must be a different digit.
- `C`, `I`, `F`, and `T` can't be zero (since we don't write leading zeros in numbers).

You can solve cryptarithmetic problems with either the new CP-SAT solver, which is more efficient, or the original CP solver. We'll show you examples using both solvers, starting with CP-SAT.

### CP-SAT Solution

In [16]:
# Declare the model
model = cp_model.CpModel()
model

When using the CP-SAT solver, there are certain helper methods it's useful to define. We'll use one of them, `NewIntVar`, to declare our (integer) digits. We distinguish between the letters that can potentially be zero and those that can't (`C`, `I`, `F`, and `T`).

In [17]:
base = 10

c = model.NewIntVar(1, base - 1, 'C')
p = model.NewIntVar(0, base - 1, 'P')
i = model.NewIntVar(1, base - 1, 'I')
s = model.NewIntVar(0, base - 1, 'S')
f = model.NewIntVar(1, base - 1, 'F')
u = model.NewIntVar(0, base - 1, 'U')
n = model.NewIntVar(0, base - 1, 'N')
t = model.NewIntVar(1, base - 1, 'T')
r = model.NewIntVar(0, base - 1, 'R')
e = model.NewIntVar(0, base - 1, 'E')

# We need to group variables in a list to use the constraint AllDifferent
letters = [c, p, i, s, f, u, n, t, r, e]

# Verify that we have enough digits
assert base >= len(letters)

Next, constraints. First, we ensure that all letters have different values, using the `AddAllDifferent` helper method. Then we use the `AddEquality` helper method to create constraints that enforce the `CP + IS + FUN = TRUE` equality.

In [18]:
model.AddAllDifferent(letters)

# CP + IS + FUN = TRUE
model.Add(c * base + p + i * base + s + f * base * base + u * base +
          n == t * base * base * base + r * base * base + u * base + e)

# Solution printer
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print('%s = %i' % (v, self.Value(v)), end='  ')
        print()

    def solution_count(self):
        return self.__solution_count 

# Invoke solver
solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter(letters)

# Enumerate all solutions
solver.parameters.enumerate_all_solutions = True

# Solve
status = solver.Solve(model, solution_printer)

# Statistics.
print('\nStatistics')
print(f'  status   : {solver.StatusName(status)}')
print(f'  conflicts: {solver.NumConflicts()}')
print(f'  branches : {solver.NumBranches()}')
print(f'  wall time: {solver.WallTime()} s')
print(f'  sol found: {solution_printer.solution_count()}')

C = 6  P = 4  I = 3  S = 5  F = 9  U = 2  N = 8  T = 1  R = 0  E = 7  
C = 3  P = 4  I = 6  S = 5  F = 9  U = 2  N = 8  T = 1  R = 0  E = 7  
C = 3  P = 4  I = 6  S = 8  F = 9  U = 2  N = 5  T = 1  R = 0  E = 7  
C = 3  P = 2  I = 6  S = 7  F = 9  U = 8  N = 5  T = 1  R = 0  E = 4  
C = 3  P = 2  I = 6  S = 5  F = 9  U = 8  N = 7  T = 1  R = 0  E = 4  
C = 2  P = 3  I = 7  S = 6  F = 9  U = 8  N = 5  T = 1  R = 0  E = 4  
C = 2  P = 3  I = 7  S = 4  F = 9  U = 6  N = 8  T = 1  R = 0  E = 5  
C = 2  P = 3  I = 7  S = 5  F = 9  U = 4  N = 8  T = 1  R = 0  E = 6  
C = 2  P = 3  I = 7  S = 8  F = 9  U = 4  N = 5  T = 1  R = 0  E = 6  
C = 2  P = 3  I = 7  S = 8  F = 9  U = 6  N = 4  T = 1  R = 0  E = 5  
C = 2  P = 3  I = 7  S = 5  F = 9  U = 8  N = 6  T = 1  R = 0  E = 4  
C = 7  P = 3  I = 2  S = 5  F = 9  U = 8  N = 6  T = 1  R = 0  E = 4  
C = 7  P = 3  I = 2  S = 6  F = 9  U = 8  N = 5  T = 1  R = 0  E = 4  
C = 6  P = 2  I = 3  S = 7  F = 9  U = 8  N = 5  T = 1  R = 0  E = 4  
C = 6 

### Original CP Solution

In this case we'll treat the base as a variable, so you can solve the equation for higher bases. (There can be no lower base solutions for `CP + IS + FUN = TRUE` since the ten letters must all be different.)

In [20]:
# Create solver
solver = pywrapcp.Solver('CP is fun!')
solver

<ortools.constraint_solver.pywrapcp.Solver; proxy of <Swig Object of type 'operations_research::Solver *' at 0x00000259E86887B0> >

The first step is to create an `IntVar` for each letter. We distinguish between the letters that can potentially be zero and those that can't (`C`, `I`, `F`, and `T`).

Next, we create an array containing a new `IntVar` for each letter. This is only necessary because when we define our constraints, we're going to use `AllDifferent`, so we need some array for which every element needs to differ.

Finally, we verify that our base is at least as large as the number of letters; otherwise, there's no solution.

In [21]:
base = 10

# Decision variables
digits = list(range(0, base))
digits_without_zero = list(range(1, base))
c = solver.IntVar(digits_without_zero, 'C')
p = solver.IntVar(digits, 'P')
i = solver.IntVar(digits_without_zero, 'I')
s = solver.IntVar(digits, 'S')
f = solver.IntVar(digits_without_zero, 'F')
u = solver.IntVar(digits, 'U')
n = solver.IntVar(digits, 'N')
t = solver.IntVar(digits_without_zero, 'T')
r = solver.IntVar(digits, 'R')
e = solver.IntVar(digits, 'E')

# We need to group variables in a list to use the constraint AllDifferent
letters = [c, p, i, s, f, u, n, t, r, e]

# Verify that we have enough digits.
assert base >= len(letters)

Now that we've defined our variables, the next step is to define constraints. First, we add the `AllDifferent` constraint, forcing each letter to have a different digit.

Next, we add the `CP + IS + FUN = TRUE` constraint. The sample programs do this in different ways.

In [22]:
solver.Add(solver.AllDifferent(letters))

# CP + IS + FUN = TRUE
solver.Add(p + s + n + base * (c + i + u) + base * base * f == e +
           base * u + base * base * r + base * base * base * t)

Now that we have our variables and constraints, we're ready to solve.

The code for the solution printer, which displays each solution as the solver finds it, is shown below.

Because there's more than one solution to our problem, we iterate through the solutions with a while `solver.NextSolution()` loop. 

In [23]:
solution_count = 0
db = solver.Phase(letters, solver.INT_VAR_DEFAULT, solver.INT_VALUE_DEFAULT)

solver.NewSearch(db)
while solver.NextSolution():
    print(letters)
    # Is CP + IS + FUN = TRUE ?
    assert (base * c.Value() + p.Value() + base * i.Value() + s.Value() +
            base * base * f.Value() + base * u.Value() +
            n.Value() == base * base * base * t.Value() +
            base * base * r.Value() + base * u.Value() + e.Value())
    solution_count += 1
solver.EndSearch()
print(f'Number of solutions found: {solution_count}')

[C(2), P(3), I(7), S(4), F(9), U(6), N(8), T(1), R(0), E(5)]
[C(2), P(3), I(7), S(5), F(9), U(4), N(8), T(1), R(0), E(6)]
[C(2), P(3), I(7), S(5), F(9), U(8), N(6), T(1), R(0), E(4)]
[C(2), P(3), I(7), S(6), F(9), U(8), N(5), T(1), R(0), E(4)]
[C(2), P(3), I(7), S(8), F(9), U(4), N(5), T(1), R(0), E(6)]
[C(2), P(3), I(7), S(8), F(9), U(6), N(4), T(1), R(0), E(5)]
[C(2), P(4), I(7), S(3), F(9), U(6), N(8), T(1), R(0), E(5)]
[C(2), P(4), I(7), S(8), F(9), U(6), N(3), T(1), R(0), E(5)]
[C(2), P(5), I(7), S(3), F(9), U(4), N(8), T(1), R(0), E(6)]
[C(2), P(5), I(7), S(3), F(9), U(8), N(6), T(1), R(0), E(4)]
[C(2), P(5), I(7), S(6), F(9), U(8), N(3), T(1), R(0), E(4)]
[C(2), P(5), I(7), S(8), F(9), U(4), N(3), T(1), R(0), E(6)]
[C(2), P(6), I(7), S(3), F(9), U(8), N(5), T(1), R(0), E(4)]
[C(2), P(6), I(7), S(5), F(9), U(8), N(3), T(1), R(0), E(4)]
[C(2), P(8), I(7), S(3), F(9), U(4), N(5), T(1), R(0), E(6)]
[C(2), P(8), I(7), S(3), F(9), U(6), N(4), T(1), R(0), E(5)]
[C(2), P(8), I(7), S(4),

## The N-queens problem

In the following sections, we'll illustrate constraint programming (CP) by a combinatorial problem based on the game of chess. In chess, a queen can attack horizontally, vertically, and diagonally. The N-queens problem asks:

> How can N queens be placed on an NxN chessboard so that no two of them attack each other?

Below, you can see one possible solution to the N-queens problem for N = 4.

<p align = 'center'>
    <img src = 'https://developers.google.com/static/optimization/images/queens/sol_4x4_b.png'>
</p>

No two queens are on the same row, column, or diagonal.

Note that this isn't an optimization problem: we want to find all possible solutions, rather than one optimal solution, which makes it a natural candidate for constraint programming. The following sections describe the CP approach to the N-queens problem, and present programs that solve it using both the CP-SAT solver and the original CP solver.

### CP approach to the N-queens problem

A CP solver works by systematically trying all possible assignments of values to the variables in a problem, to find the feasible solutions. In the 4-queens problem, the solver starts at the leftmost column and successively places one queen in each column, at a location that is not attacked by any previously placed queens.

#### Propagation and backtracking

There are two key elements to a constraint programming search:
- **Propagation** — Each time the solver assigns a value to a variable, the constraints **add restrictions on the possible values of the unassigned variables**. These restrictions propagate to future variable assignments. For example, in the 4-queens problem, each time the solver places a queen, it can't place any other queens on the row and diagonals the current queen is on. Propagation can speed up the search significantly by reducing the set of variable values the solver must explore.
- **Backtracking** occurs when either the **solver can't assign a value to the next variable, due to the constraints, or it finds a solution**. In either case, the solver backtracks to a previous stage and changes the value of the variable at that stage to a value that hasn't already been tried. In the 4-queens example, this means moving a queen to a new square on the current column.

Next, you'll see how constraint programming uses propagation and backtracking to solve the 4-queens problem. Let's suppose the solver starts by arbitrarily placing a queen in the upper left corner. That's a hypothesis of sorts; perhaps it will turn out that no solution exists with a queen in the upper left corner.

Given this hypothesis, what constraints can we propagate? One constraint is that there can be only one queen in a column (the gray Xs below), and another constraint prohibits two queens on the same diagonal (the red Xs below).

<p align = 'center'>
    <img src = 'https://developers.google.com/static/optimization/images/queens/propagation1.png'>
</p>

Our third constraint prohibits queens on the same row:

<p align = 'center'>
    <img src = 'https://developers.google.com/static/optimization/images/queens/propagation2.png'>
</p>

Our constraints propagated, we can test out another hypothesis, and place a second queen on one of the available remaining squares. Our solver might decide to place in it the first available square in the second column:

<p align = 'center'>
    <img src = 'https://developers.google.com/static/optimization/images/queens/propagation3.png'>
</p>

After propagating the diagonal constraint, we can see that it leaves no available squares in either the third column or last row:

<p align = 'center'>
    <img src = 'https://developers.google.com/static/optimization/images/queens/propagation4.png'>
</p>

With no solutions possible at this stage, we need to backtrack. One option is for the solver to choose the other available square in the second column. However, constraint propagation then forces a queen into the second row of the third column, leaving no valid spots for the fourth queen:

<p align = 'center'>
    <img src = 'https://developers.google.com/static/optimization/images/queens/propagation6.png'>
</p>

And so the solver must backtrack again, this time all the way back to the placement of the first queen. We have now shown that no solution to the queens problem will occupy a corner square.

Since there can be no queen in the corner, the solver moves the first queen down by one, and propagates, leaving only one spot for the second queen:

<p align = 'center'>
    <img src = 'https://developers.google.com/static/optimization/images/queens/propagation9.png'>
</p>

Propagating again reveals only one spot left for the third queen:

<p align = 'center'>
    <img src = 'https://developers.google.com/static/optimization/images/queens/propagation10.png'>
</p>

And for the fourth and final queen:

<p align = 'center'>
    <img src = 'https://developers.google.com/static/optimization/images/queens/propagation11.png'>
</p>

We have our first solution! If we instructed our solver to stop after finding the first solution, it would end here. Otherwise, it would backtrack again and place the first queen in the third row of the first column.

#### Solution using CP-SAT

In [34]:
# Declare the model
model = cp_model.CpModel()

# Declare board size
board_size = 8

# The array index is the column, and the value is the row
queens = [
    model.NewIntVar(0, board_size - 1, 'x%i' % i) for i in range(board_size)
]

Here we assume that `queens[j]` is the row number for the queen in column `j`. In other words, `queens[j] = i` means there is a queen in row `i` and column `j`.

For constraints, we use the `AddAllDifferent` method, which requires all the elements of a variable array to be different.

Let's see how these constraints guarantee the three conditions for the N-queens problem (queens on different rows, columns, and diagonals).
- **No two queens on the same row**: Applying the solver's `AllDifferent` method to queens forces the values of `queens[j]` to be different for each j, which means that all queens must be in different rows.
- **No two queens on the same column**: This constraint is implicit in the definition of `queens`. Since no two elements of `queens` can have the same index, no two queens can be in the same column.
- **No two queens on the same diagonal**: The diagonal constraint is a little trickier than the row and column constraints. 
    - First, if two queens lie on the same diagonal, one of the following conditions must be true:
        - The row number plus the column number for each of the two queens are equal. In other words, `queens(j) + j` has the same value for two different indices `j`.
        - The row number minus the column number for each of the two queens are equal. In this case, `queens(j) - j` has the same value for two different indices `j`.
    - One of these conditions means the queens lie on the same ascending diagonal ( going from left to right), while the other means they lie on the same descending diagonal. Which condition corresponds to ascending and which to descending depends on how you order the rows and columns. The ordering has no effect on the set of solutions, just on how you visualize them.
    - So the diagonal constraint is that the values of `queens(j) + j` must all be different, and the values of `queens(j) - j` must all be different, for different `j`.

In [35]:
# Create constraints
# All rows must be different
model.AddAllDifferent(queens)

# All columns must be different because the indices of the queens are all different

# No two queens can be on the same diagonal
model.AddAllDifferent(queens[i] + i for i in range(board_size))
model.AddAllDifferent(queens[i] - i for i in range(board_size))

To print all solutions to the N-queens problem, we need to pass a callback to the CP-SAT solver. Let's make a class.

In [40]:
class NQueenSolutionPrinter(cp_model.CpSolverSolutionCallback):
    '''Print intermediate solutions.'''

    def __init__(self, queens):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__queens = queens
        self.__solution_count = 0
        self.__start_time = time.time()
    
    def solution_count(self):
        return self.__solution_count

    def on_solution_callback(self):
        current_time = time.time()
        print('Solution %i, time = %f s' %
                (self.__solution_count, current_time - self.__start_time))
        self.__solution_count += 1

        all_queens = range(len(self.__queens))
        for i in all_queens:
            for j in all_queens:
                if self.Value(self.__queens[j]) == i:
                    # There is a queen in column j, row i
                    print('Q', end = '  ')
                else:
                    print('_', end = '  ')
            print()
        print()

In [41]:
# Call solver
solver = cp_model.CpSolver()

# Display solutions
solution_printer = NQueenSolutionPrinter(queens)
solver.parameters.enumerate_all_solutions = True
solver.Solve(model, solution_printer)

# Statistics.
print('\nStatistics')
print(f'  conflicts      : {solver.NumConflicts()}')
print(f'  branches       : {solver.NumBranches()}')
print(f'  wall time      : {solver.WallTime()} s')
print(f'  solutions found: {solution_printer.solution_count()}')

Solution 0, time = 0.006000 s
_  _  _  Q  _  _  _  _  
_  _  _  _  _  _  Q  _  
_  _  _  _  Q  _  _  _  
_  _  Q  _  _  _  _  _  
Q  _  _  _  _  _  _  _  
_  _  _  _  _  Q  _  _  
_  _  _  _  _  _  _  Q  
_  Q  _  _  _  _  _  _  

Solution 1, time = 0.009535 s
_  _  _  _  _  _  _  Q  
_  Q  _  _  _  _  _  _  
_  _  _  _  Q  _  _  _  
_  _  Q  _  _  _  _  _  
Q  _  _  _  _  _  _  _  
_  _  _  _  _  _  Q  _  
_  _  _  Q  _  _  _  _  
_  _  _  _  _  Q  _  _  

Solution 2, time = 0.010541 s
_  _  _  Q  _  _  _  _  
_  _  _  _  _  _  _  Q  
_  _  _  _  Q  _  _  _  
_  _  Q  _  _  _  _  _  
Q  _  _  _  _  _  _  _  
_  _  _  _  _  _  Q  _  
_  Q  _  _  _  _  _  _  
_  _  _  _  _  Q  _  _  

Solution 3, time = 0.013541 s
_  _  _  _  _  _  Q  _  
_  Q  _  _  _  _  _  _  
_  _  _  _  _  Q  _  _  
_  _  Q  _  _  _  _  _  
Q  _  _  _  _  _  _  _  
_  _  _  Q  _  _  _  _  
_  _  _  _  _  _  _  Q  
_  _  _  _  Q  _  _  _  

Solution 4, time = 0.017562 s
_  _  _  Q  _  _  _  _  
_  _  _  _  _  Q  _  

### Solve using original CP solver

In [42]:
# Declare solver
solver = pywrapcp.Solver('n-queens')

# Declare board size
board_size = 8

# Create variables
queens = [solver.IntVar(0, board_size - 1, f'x{i}') for i in range(board_size)]

# Create constraints
# All rows must be different
solver.Add(solver.AllDifferent(queens))

# No two queens can be on the same diagonal
solver.Add(solver.AllDifferent([queens[i] + i for i in range(board_size)]))
solver.Add(solver.AllDifferent([queens[i] - i for i in range(board_size)]))

Let's see how these constraints guarantee the three conditions for the N-queens problem (queens on different rows, columns, and diagonals).
- **No two queens on the same row**: Applying the solver's `AllDifferent` method to queens forces the values of `queens[j]` to be different for each j, which means that all queens must be in different rows.
- **No two queens on the same column**: This constraint is implicit in the definition of `queens`. Since no two elements of `queens` can have the same index, no two queens can be in the same column.
- **No two queens on the same diagonal**: The diagonal constraint is a little trickier than the row and column constraints. 
    - First, if two queens lie on the same diagonal, one of the following must be true:
        - If the diagonal is descending (going from left to right), the row number plus column number for each of the two queens are equal. So `queens(i) + i` has the same value for two different indices `i`.
        - If the diagonal is ascending, the row number minus the column number for each of the two queens are equal. In this case, `queens(i) - i` has the same value for two different indices `i`.
    - So the diagonal constraint is that the values of `queens(i) + i` must all be different, and likewise the values of `queens(i) - i` must all be different, for different `i`.
    - The code above adds this constraints by applying the `AllDifferent` method to `queens[j] + j` and `queens[j] - j`, for each `i`.

In [43]:
db = solver.Phase(queens, solver.CHOOSE_FIRST_UNBOUND, solver.ASSIGN_MIN_VALUE)

After making the constraints, the next step is to create a decision builder, which sets the search strategy for the problem. The search strategy can have a major impact on the search time, due to propagation of constraints, which reduces the number of variable values the solver has to explore. You have already seen an example of this in the 4-queens example.

Let's take a look at how decision builder directs the search in the 4-queens example. The solver begins with `queens[0]`, the first variable in the array, as directed by `CHOOSE_FIRST_UNBOUND`. The solver then assigns `queens[0]` the smallest value that hasn't already been tried, which is 0 at this stage, as directed by `ASSIGN_MIN_VALUE`. This places the first queen in the upper left corner of the board.

Next, the solver selects `queens[1]`, which is now the first unbound variable in queens. After propagating the constraints, there are two possible rows for a queen in column 1: row 2 or row 3. The `ASSIGN_MIN_VALUE` option directs the solver to assign `queens[1] = 2`. (If instead, you set `IntValueStrategy` to `ASSIGN_MAX_VALUE`, the solver would assign `queens[1] = 3`.)

You can check that the rest of the search follows the same rules.

In [44]:
# Display solutions
num_solutions = 0
solver.NewSearch(db)
while solver.NextSolution():
    # Displays the solution just computed
    for i in range(board_size):
        for j in range(board_size):
            if queens[j].Value() == i:
                # There is a queen in column j, row i
                print('Q', end = '  ')
            else:
                print('_', end = '  ')
        print()
    print()
    num_solutions += 1
solver.EndSearch()

# Statistics
print('\nStatistics')
print(f'  failures: {solver.Failures()}')
print(f'  branches: {solver.Branches()}')
print(f'  wall time: {solver.WallTime()} ms')
print(f'  Solutions found: {num_solutions}')

Q  _  _  _  _  _  _  _  
_  _  _  _  _  _  Q  _  
_  _  _  _  Q  _  _  _  
_  _  _  _  _  _  _  Q  
_  Q  _  _  _  _  _  _  
_  _  _  Q  _  _  _  _  
_  _  _  _  _  Q  _  _  
_  _  Q  _  _  _  _  _  

Q  _  _  _  _  _  _  _  
_  _  _  _  _  _  Q  _  
_  _  _  Q  _  _  _  _  
_  _  _  _  _  Q  _  _  
_  _  _  _  _  _  _  Q  
_  Q  _  _  _  _  _  _  
_  _  _  _  Q  _  _  _  
_  _  Q  _  _  _  _  _  

Q  _  _  _  _  _  _  _  
_  _  _  _  _  Q  _  _  
_  _  _  _  _  _  _  Q  
_  _  Q  _  _  _  _  _  
_  _  _  _  _  _  Q  _  
_  _  _  Q  _  _  _  _  
_  Q  _  _  _  _  _  _  
_  _  _  _  Q  _  _  _  

Q  _  _  _  _  _  _  _  
_  _  _  _  Q  _  _  _  
_  _  _  _  _  _  _  Q  
_  _  _  _  _  Q  _  _  
_  _  Q  _  _  _  _  _  
_  _  _  _  _  _  Q  _  
_  Q  _  _  _  _  _  _  
_  _  _  Q  _  _  _  _  

_  _  _  _  _  Q  _  _  
Q  _  _  _  _  _  _  _  
_  _  _  _  Q  _  _  _  
_  Q  _  _  _  _  _  _  
_  _  _  _  _  _  _  Q  
_  _  Q  _  _  _  _  _  
_  _  _  _  _  _  Q  _  
_  _  _  Q  _  _  _  

# Setting solver limits

## Setting a time limit for the solver

If your program takes a long time to run, we recommend setting a time limit for the solver, which ensures that the program will terminate in a reasonable length of time. The examples below illustrate how to set a limit of 10 seconds for the solver.

In [45]:
'''Solves a problem with a time limit.'''

def SolveWithTimeLimitSampleSat():
    '''Minimal CP-Sat example to showcase calling the solver.'''
    
    # Create the model
    model = cp_model.CpModel()

    # Create variables
    num_vals = 3
    x = model.NewIntVar(0, num_vals - 1, 'x')
    y = model.NewIntVar(0, num_vals - 1, 'y')
    z = model.NewIntVar(0, num_vals - 1, 'z')
    # Adds an all-different constraint
    model.Add(x != y)

    # Create a solver
    solver = cp_model.CpSolver()

    # Sets a time limit of 10 seconds
    solver.parameters.max_time_in_seconds = 10.0

    status = solver.Solve(model)

    if status == cp_model.OPTIMAL:
        print('x = %i' % solver.Value(x))
        print('y = %i' % solver.Value(y))
        print('z = %i' % solver.Value(z))

SolveWithTimeLimitSampleSat()

x = 1
y = 0
z = 0


## Stopping a search after a specified number of solutions

As an alternative to setting a time limit, you can make the solver terminate after it finds a specified number of solutions. 

In [49]:
'''Code sample that solves a model and displays a small number of solutions.'''

class VarArraySolutionPrinterWithLimit(cp_model.CpSolverSolutionCallback):
    '''Prints intermediate solutions.'''

    def __init__(self, variables, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0
        self.__solution_limit = limit
    
    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print('%s = %i' % (v, self.Value(v)), end = '  ')
        print()
        if self.__solution_count >= self.__solution_limit:
            print('Stop search after %i solutions' % self.__solution_limit)
            self.StopSearch()

    def solution_count(self):
        return self.__solution_count

def StopAfterNSolutionsSampleSat(n = 5):
    '''Showcases calling the solver to search for small number of solutions.'''

    # Creates the model
    model = cp_model.CpModel()

    # Creates the variables
    num_vals = 3
    x = model.NewIntVar(0, num_vals - 1, 'x')
    y = model.NewIntVar(0, num_vals - 1, 'y')
    z = model.NewIntVar(0, num_vals - 1, 'z')

    # Create a solver and solve
    solver = cp_model.CpSolver()
    solution_printer = VarArraySolutionPrinterWithLimit([x, y, z], n)

    # Enumerate all solutions
    solver.parameters.enumerate_all_solutions = True
    
    # Solve
    status = solver.Solve(model, solution_printer)

    print('Status = %s' % solver.StatusName(status))
    print('Number of solutions found: %i' % solution_printer.solution_count())
    assert solution_printer.solution_count() <= n

StopAfterNSolutionsSampleSat(8)

x = 0  y = 0  z = 0  
x = 0  y = 1  z = 0  
x = 0  y = 1  z = 1  
x = 0  y = 0  z = 1  
x = 0  y = 2  z = 1  
x = 0  y = 2  z = 0  
x = 0  y = 2  z = 2  
x = 0  y = 1  z = 2  
Stop search after 8 solutions
Status = FEASIBLE
Number of solutions found: 8
